## Installing and Importing dependencies

In [ ]:
!pip install -q crewai crewai-tools langchain-google-genai langchain-community
import os
from crewai import Agent, Task, Crew
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Dict, Any

## Set up Gemini API


In [ ]:
os.environ["GOOGLE_API_KEY"] = "" ## removing the key from here for security
llm = ChatGoogleGenerativeAI(
    model="gemini/gemini-2.0-flash",
    temperature=0.7,
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

## Create a simple Wikipedia search function


In [ ]:
def search_wikipedia(search_query: str) -> str:
    from langchain_community.utilities import WikipediaAPIWrapper
    wiki = WikipediaAPIWrapper()
    try:
        result = wiki.run(search_query)
        return result[:1000]
    except Exception as e:
        return f"Error searching Wikipedia: {str(e)}"

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:1037: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  for field_name, field_info in self.model_fields.items():


##  Memory-based Agent in CrewAI

#### Create a memory-aware agent


In [ ]:
llm = LLM(
    model="gemini/gemini-2.0-flash",
    api_key=os.environ["GOOGLE_API_KEY"]
)
memory_agent = Agent(
    role="Memory Assistant",
    goal="Maintain conversation context and provide relevant responses based on previous interactions",
    backstory="""You are a helpful assistant with excellent memory. You track and remember details from
    conversations to provide contextual and personalized responses.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

## Create a conversation task with memory


In [ ]:
conversation_task = Task(
    description="""
    You are having a conversation. In the first exchange, I will tell you "My favorite color is blue. Remember this!"
    Then you need to respond recognizing this information.

    In the second exchange, I will ask "What's my favorite color?" and you should recall the color blue from our previous exchange.

    This tests your ability to maintain memory across exchanges.
    """,
    expected_output="Appropriate response showing memory of the favorite color mentioned earlier",
    agent=memory_agent
)

## Create and execute crew

In [ ]:
memory_crew = Crew(
    agents=[memory_agent],
    tasks=[conversation_task],
    verbose=True
)
memory_result = memory_crew.kickoff()
print("\nMemory Agent Results:")
print(memory_result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 941bbeab-a360-45c1-bf76-c833ac678c14                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 91a36e08-c439-4c6f-9347-191922039dfc
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 91a36e08-c439-4c6f-9347-191922039dfc
       Status: Executing Task...
    └── 🤖 Agent: Memory Assistant
            Status: In Progress

# Agent: Memory Assistant
## Task: 
    You are having a conversation. In the first exchange, I will tell you "My favorite color is blue. Remember this!"
    Then you need to respond recognizing this information.
    
    In the second exchange, I will ask "What's my favorite color?" and you should recall the color blue from our previous exchange.
    
    This tests your ability to maintain memory across exchanges.
    


# Agent: Memory Assistant
## Final Answer: 
Okay, I will remember that your favorite color is blue!




🚀 Crew: crew
└── 📋 Task: 91a36e08-c439-4c6f-9347-191922039dfc
       Status: Executing Task...
    └── 🤖 Agent: Memory Assistant
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 91a36e08-c439-4c6f-9347-191922039dfc
       Assigned to: Memory Assistant
       Status: ✅ Completed
    └── 🤖 Agent: Memory Assistant
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 91a36e08-c439-4c6f-9347-191922039dfc                                                                     │
│  Agent: Memory Assistant                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 941bbeab-a360-45c1-bf76-c833ac678c14                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Memory Agent Results:
Okay, I will remember that your favorite color is blue!


## 2. Tool-integrated Agent

This agent demonstrates how to integrate external tools in CrewAI. The agent can access Wikipedia to search for information when needed.

Key features:
- Tool integration
- Automatic tool selection  
- Function-based tools
- Dynamic information retrieval

In [ ]:
# Create a tool-using agent with Wikipedia search capability
from langchain_community.utilities import WikipediaAPIWrapper

# Initialize Wikipedia utility
wiki_util = WikipediaAPIWrapper()

# Create tool-integrated agent
tool_agent = Agent(
   role="Research Assistant",
   goal="Find accurate information using available tools to answer user questions",
   backstory="""You are a research assistant who can access Wikipedia to find information.
   When a user asks for information, you should search Wikipedia and provide relevant answers.""",
   verbose=True,
   allow_delegation=False,
   llm=llm
)

# Create a simple tool-using task
search_task = Task(
   description="""Search for information about "Python programming language" and provide a brief summary.""",
   expected_output="Summary of information found about Python programming language",
   agent=tool_agent
)

# Test tool usage with direct search function
def execute_search_tool(query):
   try:
       result = wiki_util.run(query)
       return result[:800]  # Limit response length
   except Exception as e:
       return f"Search error: {str(e)}"

# Since CrewAI doesn't easily support custom tools with Gemini,
# we'll create a workaround agent that "uses tools" via task description
tool_task = Task(
   description="""You are a research agent with access to Wikipedia search.
   A user wants to know about "Artificial Intelligence history".

   Step 1: Search Wikipedia for "Artificial Intelligence history"
   Step 2: Summarize the key historical developments in AI
   Step 3: Provide a concise overview of major milestones

   Respond as if you've searched and found information about AI history.""",
   expected_output="Summary of Artificial Intelligence history from Wikipedia",
   agent=tool_agent
)

# Execute search crew
tool_crew = Crew(
   agents=[tool_agent],
   tasks=[tool_task],
   verbose=True
)

print("Executing tool agent...")
tool_result = tool_crew.kickoff()
print("\nTool Agent Results:")
print(tool_result)

# Alternative demonstration: directly search and format response
print("\n--- Direct Wikipedia Search for Reference ---")
search_result = execute_search_tool("Python programming language")
print(f"Direct search result: {search_result}")

Executing tool agent...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 622054c3-27ee-4201-aa9e-4d3fc3027317                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 821c2653-401a-489f-938c-52ac44243402
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 821c2653-401a-489f-938c-52ac44243402
       Status: Executing Task...
    └── 🤖 Agent: Research Assistant
            Status: In Progress

# Agent: Research Assistant
## Task: You are a research agent with access to Wikipedia search. 
   A user wants to know about "Artificial Intelligence history". 
   
   Step 1: Search Wikipedia for "Artificial Intelligence history"
   Step 2: Summarize the key historical developments in AI
   Step 3: Provide a concise overview of major milestones
   
   Respond as if you've searched and found information about AI history.
 Received None or empty response from LLM call.
 An unknown error occurred. Please check the details below.
 Error details: Invalid response from LLM call - None or empty.
 An unknown error occurred. Please check the details below.
 Error details: Invalid response from LLM call - None or empty.


...

🚀 Crew: crew
└── 📋 Task: 821c2653-401a-489f-938c-52ac44243402
       Status: Executing Task...
    ├── 🤖 Agent: Research Assistant
    │       Status: In Progress
    └── 🤖 Agent: Research Assistant
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 821c2653-401a-489f-938c-52ac44243402
       Assigned to: Research Assistant
       Status: ✅ Completed
    ├── 🤖 Agent: Research Assistant
    │       Status: In Progress
    └── 🤖 Agent: Research Assistant
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 821c2653-401a-489f-938c-52ac44243402                                                                     │
│  Agent: Research Assistant                                                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 622054c3-27ee-4201-aa9e-4d3fc3027317                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Tool Agent Results:
Artificial intelligence (AI) has a long history, dating back to ancient times with myths, stories and rumors of artificial beings endowed with intelligence or consciousness by master craftsmen. After World War II, a few scientists began to independently explore the possibility of building an artificial brain.

The field of AI research was founded at a workshop held on the campus of Dartmouth College in the summer of 1956. Attendees Allen Newell, Cliff Shaw, Herbert Simon, Arthur Samuel, and John McCarthy would become the leaders of AI research for many years. They and their students wrote programs that were, to most people, simply astonishing: computers were solving algebra problems, proving logical theorems, and learning to speak English. By the middle of the 1960s, research in the U.S. was heavily funded by the Department of Defense and laboratories had been established around the world.

AI researchers predicted that truly intelligent machines would be possible 

In [ ]:
# Create specialized agents for collaboration
researcher_agent = Agent(
   role="Research Analyst",
   goal="Gather and analyze information on requested topics",
   backstory="""You are an experienced research analyst who specializes in collecting
   and analyzing information from various sources. You provide accurate, concise summaries.""",
   verbose=True,
   allow_delegation=False,
   llm=llm
)

writer_agent = Agent(
   role="Content Writer",
   goal="Transform research findings into well-written content",
   backstory="""You are a skilled content writer who takes research data and creates
   engaging, informative articles. You focus on clarity and readability.""",
   verbose=True,
   allow_delegation=False,
   llm=llm
)

editor_agent = Agent(
   role="Content Editor",
   goal="Review and refine written content for quality and accuracy",
   backstory="""You are an experienced editor who ensures content is accurate, well-structured,
   and free of errors. You provide constructive feedback and improvements.""",
   verbose=True,
   allow_delegation=False,
   llm=llm
)

# Define collaborative tasks
research_task = Task(
   description="""Research comprehensive information about "Cloud Computing Services":
   1. Define what cloud computing is
   2. List major cloud service providers
   3. Explain types of cloud services (IaaS, PaaS, SaaS)
   4. Describe benefits and challenges""",
   expected_output="Detailed research findings about cloud computing services",
   agent=researcher_agent
)

writing_task = Task(
   description="""Using the research provided, write an informative article about cloud computing:
   1. Start with an engaging introduction
   2. Explain the key concepts clearly
   3. Provide examples and comparisons
   4. Conclude with future outlook""",
   expected_output="Well-written article about cloud computing services",
   agent=writer_agent,
   dependencies=[research_task]  # This task depends on research_task completion
)

editing_task = Task(
   description="""Review and edit the article about cloud computing:
   1. Check for clarity and flow
   2. Ensure technical accuracy
   3. Improve sentence structure
   4. Add section headings if missing
   5. Finalize the content""",
   expected_output="Polished, publication-ready article about cloud computing",
   agent=editor_agent,
   dependencies=[writing_task]  # This task depends on writing_task completion
)

# Create collaborative crew
collaborative_crew = Crew(
   agents=[researcher_agent, writer_agent, editor_agent],
   tasks=[research_task, writing_task, editing_task],
   verbose=True
)

# Execute collaborative workflow
print("Starting collaborative crew workflow...")
collaboration_result = collaborative_crew.kickoff()
print("\nCollaborative Agent Results:")
print(collaboration_result)

Starting collaborative crew workflow...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 902278e0-a5b0-4aa2-afec-cae9ab4f4770                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 5d4329e1-0989-49c3-8dca-339d45656f8f
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 5d4329e1-0989-49c3-8dca-339d45656f8f
       Status: Executing Task...
    └── 🤖 Agent: Research Analyst
            Status: In Progress

# Agent: Research Analyst
## Task: Research comprehensive information about "Cloud Computing Services":
   1. Define what cloud computing is
   2. List major cloud service providers
   3. Explain types of cloud services (IaaS, PaaS, SaaS)
   4. Describe benefits and challenges


# Agent: Research Analyst
## Final Answer: 
## Cloud Computing Services: A Comprehensive Overview

**1. Definition of Cloud Computing:**

Cloud computing is the on-demand availability of computer system resources, especially data storage (cloud storage) and computing power, without direct active management by the user. The term is generally used to describe data centers available to many users over the Internet. Large clouds, predominant today, often have functions distributed over multiple locations from central servers. If the service is provided according to users' requirements, we call it cloud computing.

Cloud computing allows companies and individuals to consume computing resources as a utility – just li

...

🚀 Crew: crew
├── 📋 Task: 5d4329e1-0989-49c3-8dca-339d45656f8f
│      Assigned to: Research Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Analyst
│           Status: ✅ Completed
├── 📋 Task: d47ca9a2-c36e-49b1-9b6d-0d8d5e2384e3
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 61058d3b-c6d8-41a9-ab11-8e448603865c
       Status: Executing Task...
    └── 🤖 Agent: Content Editor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 5d4329e1-0989-49c3-8dca-339d45656f8f
│      Assigned to: Research Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Research Analyst
│           Status: ✅ Completed
├── 📋 Task: d47ca9a2-c36e-49b1-9b6d-0d8d5e2384e3
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 61058d3b-c6d8-41a9-ab11-8e448603865c
       Assigned to: Content Editor
       Status: ✅ Completed
    └── 🤖 Agent: Content Editor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 61058d3b-c6d8-41a9-ab11-8e448603865c                                                                     │
│  Agent: Content Editor                                                                                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 902278e0-a5b0-4aa2-afec-cae9ab4f4770                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Collaborative Agent Results:
## Unlock the Power of the Cloud: A Comprehensive Guide to Cloud Computing

Imagine accessing limitless computing power, storage, and software without the burden of managing complex infrastructure. This is the promise of cloud computing, a transformative technology that has reshaped how businesses and individuals interact with technology. This article explores the core concepts of cloud computing, examines different service models, and discusses the benefits and challenges of adopting this innovative approach.

### What is Cloud Computing? Demystifying the Core Concepts

At its core, cloud computing is the on-demand delivery of computing services—including servers, storage, databases, networking, software, analytics, and intelligence—over the Internet ("the cloud"). It offers faster innovation, flexible resources, and economies of scale. Instead of owning and maintaining physical data centers, users can access these resources from a cloud provider and pay 

## 4. Planning Agent System

### Create planning agents


In [ ]:
planning_agent = Agent(
   role="Project Planner",
   goal="Break down complex projects into actionable steps",
   backstory="""You are an expert project planner who excels at decomposing complex tasks
   into manageable steps. You create detailed, logical plans that ensure successful project completion.""",
   verbose=True,
   allow_delegation=False,
   llm=llm
)
execution_agent = Agent(
   role="Task Executor",
   goal="Execute tasks according to the plan effectively",
   backstory="""You are a reliable task executor who follows plans precisely and
   delivers results efficiently. You ensure each step is completed properly before moving to the next.""",
   verbose=True,
   allow_delegation=False,
   llm=llm
)

### Define planning tasks


In [ ]:
breakdown_task = Task(
   description="""Create a detailed plan for building a simple e-commerce website:
   1. Break down the project into major phases
   2. List specific tasks for each phase
   3. Define dependencies between tasks
   4. Estimate effort for each component
   5. Create a logical execution sequence""",
   expected_output="Detailed project plan with tasks and execution order",
   agent=planning_agent
)

execution_simulation_task = Task(
   description="""Following the plan created, simulate the execution process:
   1. Execute the first 3 steps from the plan
   2. Document progress and completion status
   3. Identify any potential blockers
   4. Provide status updates for each step""",
   expected_output="Execution progress report with completed tasks and status",
   agent=execution_agent,
   dependencies=[breakdown_task]
)

### Create planning crew


In [ ]:
planning_crew = Crew(
   agents=[planning_agent, execution_agent],
   tasks=[breakdown_task, execution_simulation_task],
   verbose=True
)
print("Starting planning crew workflow...")
planning_result = planning_crew.kickoff()
print("\nPlanning Agent Results:")
print(planning_result)

Starting planning crew workflow...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0c56e9b2-464a-423e-befc-198abf973f00                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 0d3c0487-2bf4-4480-8f84-d1af1b5e9d38
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 0d3c0487-2bf4-4480-8f84-d1af1b5e9d38
       Status: Executing Task...
    └── 🤖 Agent: Project Planner
            Status: In Progress

# Agent: Project Planner
## Task: Create a detailed plan for building a simple e-commerce website:
   1. Break down the project into major phases
   2. List specific tasks for each phase
   3. Define dependencies between tasks
   4. Estimate effort for each component
   5. Create a logical execution sequence


# Agent: Project Planner
## Final Answer: 
## E-commerce Website Project Plan

**1. Project Goal:**

To develop and launch a functional e-commerce website capable of showcasing products, processing orders, and managing customer accounts.

**2. Major Phases:**

1.  **Planning & Requirements Gathering:** Defining the scope, features, and functionalities of the website.
2.  **Design & Prototyping:** Creating the user interface (UI) and user experience (UX) design, and developing interactive prototypes.
3.  **Development:** Building the front-end and back-end components of the website.
4.  **Testing & Quality Assurance:** Thoroughly testing the website to identify and fix bugs.
5.  *

...

🚀 Crew: crew
├── 📋 Task: 0d3c0487-2bf4-4480-8f84-d1af1b5e9d38
│      Assigned to: Project Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Project Planner
│           Status: ✅ Completed
└── 📋 Task: 8b97d6b6-13f1-4a03-86ff-9742e105efc9
       Status: Executing Task...
    └── 🤖 Agent: Task Executor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 0d3c0487-2bf4-4480-8f84-d1af1b5e9d38
│      Assigned to: Project Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Project Planner
│           Status: ✅ Completed
└── 📋 Task: 8b97d6b6-13f1-4a03-86ff-9742e105efc9
       Assigned to: Task Executor
       Status: ✅ Completed
    └── 🤖 Agent: Task Executor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8b97d6b6-13f1-4a03-86ff-9742e105efc9                                                                     │
│  Agent: Task Executor                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0c56e9b2-464a-423e-befc-198abf973f00                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Planning Agent Results:
Execution Progress Report:

Phase 1: Planning & Requirements Gathering

Task 1.1: Define Project Scope
Description: Clearly define the boundaries of the project. What is included and what is excluded?
Status: Completed. The project scope has been defined to include the development of a functional e-commerce website with product showcasing, order processing, and customer account management. The scope excludes mobile app development and integration with third-party marketing platforms in the initial launch.
Effort: 4 hours.
Potential Blockers: None identified.
Task 1.2: Identify Target Audience
Description: Determine the demographics and characteristics of the intended users.
Status: Completed. The target audience has been identified as tech-savvy individuals aged 25-45, with an interest in online shopping and a preference for user-friendly interfaces. Further segmentation may be required based on specific product categories.
Effort: 4 hours.
Potential Blockers: 